# Amazon SageMaker Workshop
### _**Evaluation**_

---
In this part of the workshop we will get the previous model we trained to Predict Mobile Customer Departure and evaluate its performance with a test dataset.

---

## Contents

1. [Background](#Background) - Getting the model trained in the previous lab.
2. [Evaluate](#Evaluate)
    * Creating a script to evaluate model
    * Using [SageMaker Processing](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html) jobs to automate evaluation of models
3. [Exercise](#a_Exercise) - customizing metrics and evaluation reports
4. [Wrap-up - end of Evaluation Lab](#Wrap-up)


---

## Background

In the previous [Modeling](../2-Modeling/modeling.ipynb) lab we used SageMaker trained models by creating multiple SageMaker training jobs.

Install and import some packages we'll need for this lab:

In [ ]:
import boto3
import sagemaker
from sagemaker.s3 import S3Uploader, S3Downloader

In [ ]:
sm_sess = sagemaker.session.Session()
role = sagemaker.get_execution_role()

Get the variables from initial setup:

In [ ]:
%store -r bucket
%store -r prefix
%store -r region
%store -r docker_image_name

In [ ]:
bucket, prefix, region, docker_image_name

---
### - if you _**skipped**_ the data preparation lab follow instructions:

   - **run this [notebook](./config/pre_setup.ipynb)**
   - load the model S3 URI:

In [ ]:
# # Uncomment if you skipped the data preparation lab

# %store -r s3uri_model
# !cp config/model.tar.gz ./

# %store -r s3uri_test

---
### - if you _**have done**_ the previous labs

Download the model and test data from S3:

In [ ]:
# # Uncomment if you have done the previous labs

# # Get name of training job and other variables
# %store -r training_job_name
# training_job_name

In [ ]:
# # Uncomment if you have done the previous labs
# estimator = sagemaker.estimator.Estimator.attach(training_job_name)
# s3uri_model = estimator.model_data
# print("\ns3uri_model =",s3uri_model)

# S3Downloader.download(s3uri_model, ".")

In [ ]:
# %store -r s3uri_test
# S3Downloader.download(s3uri_test, ".")

---

Now you should have the `model.tar.gz` file in the 3-Evaluation directory 

(click the refresh button)

![refresh_dir.png](./media/refresh_dir.png)

# Evaluate model

Let's create a simple evaluation with some Scikit-Learn Metrics like [Area Under the Curve (AUC)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html) and [Accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html).

In [ ]:
import json
import os
import tarfile
import logging
import pickle

import pandas as pd
import xgboost

from sklearn.metrics import classification_report, roc_auc_score, accuracy_score


model_path = "model.tar.gz"
with tarfile.open(model_path) as tar:
    tar.extractall(path=".")

print("Loading xgboost model.")
model = pickle.load(open("xgboost-model", "rb"))
model

In [ ]:
print("Loading test input data")
test_path = "test.csv"
df = pd.read_csv(test_path, header=None)
df

In [ ]:
print("Reading test data. We should get an `DMatrix` object...")
y_test = df.iloc[:, 0].to_numpy()
df.drop(df.columns[0], axis=1, inplace=True)
X_test = xgboost.DMatrix(df.values)
X_test

In [ ]:
print("Performing predictions against test data.")
predictions_probs = model.predict(X_test)
predictions = predictions_probs.round()

print("Creating classification evaluation report")
acc = accuracy_score(y_test, predictions)
auc = roc_auc_score(y_test, predictions_probs)

print("Accuracy =", acc)
print("AUC =", auc)

### Creating a classification report

Now, let's save the results in a JSON file, following the structure defined in SageMaker docs:
https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html

We'll use this logic later in [Lab 6-Pipelines](../6-Pipelines/pipelines.ipynb):

In [ ]:
import pprint
# The metrics reported can change based on the model used - check the link for the documentation 
report_dict = {
    "binary_classification_metrics": {
        "accuracy": {
            "value": acc,
            "standard_deviation": "NaN",
        },
        "auc": {"value": auc, "standard_deviation": "NaN"},
    },
}

print("Classification report:")
pprint.pprint(report_dict)

In [ ]:
evaluation_output_path = os.path.join(
    ".", "evaluation.json"
)
print("Saving classification report to {}".format(evaluation_output_path))

with open(evaluation_output_path, "w") as f:
    f.write(json.dumps(report_dict))

---

## Ok, now we have working code. Let's put it in a Python Script

In [ ]:
%%writefile evaluate.py
"""Evaluation script for measuring model accuracy."""

import json
import os
import tarfile
import logging
import pickle

import pandas as pd
import xgboost

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

# May need to import additional metrics depending on what you are measuring.
# See https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

def get_dataset(dir_path, dataset_name) -> pd.DataFrame:
    files = [ os.path.join(dir_path, file) for file in os.listdir(dir_path) ]
    if len(files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(files, dataset_name))
    raw_data = [ pd.read_csv(file, header=None) for file in files ]
    df = pd.concat(raw_data)
    return df

if __name__ == "__main__":
    model_path = "/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path="..")

    logger.debug("Loading xgboost model.")
    model = pickle.load(open("xgboost-model", "rb"))

    logger.info("Loading test input data")
    test_path = "/opt/ml/processing/test"
    df = get_dataset(test_path, "test_set")

    logger.debug("Reading test data.")
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    X_test = xgboost.DMatrix(df.values)

    logger.info("Performing predictions against test data.")
    predictions_probs = model.predict(X_test)
    predictions = predictions_probs.round()

    logger.info("Creating classification evaluation report")
    acc = accuracy_score(y_test, predictions)
    auc = roc_auc_score(y_test, predictions_probs)

    # The metrics reported can change based on the model used, but it must be a specific name per (https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)
    report_dict = {
        "binary_classification_metrics": {
            "accuracy": {
                "value": acc,
                "standard_deviation": "NaN",
            },
            "auc": {"value": auc, "standard_deviation": "NaN"},
        },
    }

    logger.info("Classification report:\n{}".format(report_dict))

    evaluation_output_path = os.path.join(
        "/opt/ml/processing/evaluation", "evaluation.json"
    )
    logger.info("Saving classification report to {}".format(evaluation_output_path))

    with open(evaluation_output_path, "w") as f:
        f.write(json.dumps(report_dict))


---

## Ok, now we are finally running this script with a simple call to SageMaker Processing!

In [ ]:
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)

In [ ]:
# Processing step for evaluation
processor = ScriptProcessor(
    image_uri=docker_image_name,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="CustomerChurn/eval-script",
    sagemaker_session=sm_sess,
    role=role,
)

In [ ]:
entrypoint = "evaluate.py"

In [ ]:
from time import strftime, gmtime
# Helper to create timestamps
create_date = lambda: strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [ ]:
processor.run(
    code=entrypoint,
    inputs=[
        sagemaker.processing.ProcessingInput(
            source=s3uri_model,
            destination="/opt/ml/processing/model",
        ),
        sagemaker.processing.ProcessingInput(
            source=s3uri_test,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="evaluation", source="/opt/ml/processing/evaluation"
        ),
    ],
    job_name=f"CustomerChurnEval-{create_date()}"
)

If everything went well, the SageMaker Processing job must have created the JSON with the evaluation report of our model and saved it in S3.

In addition, under the hood, SageMaker Processing has uploaded our `evaluate.py` script to S3. Let's check where the script was saved:

In [ ]:
for proc_in in processor.latest_job.inputs:
    if proc_in.input_name == "code":
        s3_evaluation_code_uri = proc_in.source 
        
s3_evaluation_code_uri

#### Let's store the S3 URI where our evaluation script was saved for later

In [ ]:
%store s3_evaluation_code_uri

### Let's check it the evaluation report from S3!

In [ ]:
out_s3_report_uri = processor.latest_job.outputs[0].destination
out_s3_report_uri

In [ ]:
reports_list = S3Downloader.list(out_s3_report_uri)
reports_list

In [ ]:
report = S3Downloader.read_file(reports_list[0])

print("=====Model Report====")
print(json.dumps(json.loads(report.split('\n')[0]), indent=2))

# Wrap-up

Now that we finished the **evaluation lab**, let's make everything here re-usable. It may come in handy later (spoiler alert - when creating Pipelines)...

In [ ]:
%%writefile ../6-Pipelines/my_labs_solutions/evaluation_solution.py
import sagemaker
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)

def get_evaluation_processor(docker_image_name) -> ScriptProcessor:
    
    role = sagemaker.get_execution_role()
    sm_sess = sagemaker.session.Session()

    # Processing step for evaluation
    processor = ScriptProcessor(
        image_uri=docker_image_name,
        command=["python3"],
        instance_type="ml.m5.xlarge",
        instance_count=1,
        base_job_name="CustomerChurn/eval-script",
        sagemaker_session=sm_sess,
        role=role,
    )
    
    return processor